# <h1 align="center" style="font-family:Comic Sans MS">Create & Send out Reports</h1>

## 1. Install Dependencies

In [1]:
from pathlib import Path
import pandas as pd  # pip install pandas openpyxl
import win32com.client as win32  # pip install pywin32

<p style="background:black">
<code style="background:black;color:white">C:\Users\YOUR_USERNAME> pip install pandas openpyxl
</code>
<code style="background:black;color:white">C:\Users\YOUR_USERNAME> pip install pywin32
</code>
</p>

## 2. Separate Excel file (create attachments)

In [2]:
# Locate examples files & create output directory
EXCEL_FILE_PATH = Path.cwd() / "Financial_Data.xlsx"
ATTACHMENT_DIR = Path.cwd() / "Attachments"

ATTACHMENT_DIR.mkdir(exist_ok=True)

In [3]:
# Load financial data into dataframe
data = pd.read_excel(EXCEL_FILE_PATH, sheet_name="Data")
data.head()

,Segment,Country,Product,Discount Band,Units Sold,Manufacturing Price,Sale Price,Gross Sales,Discounts,Sales,COGS,Profit,Date,Month Number,Month Name,Year
0,Government,Canada,Carretera,None,1618.5,3,20,32370.0,0.0,32370.0,16185.0,16185.0,2021-01-01,1,January,2021
1,Government,Germany,Carretera,None,1321.0,3,20,26420.0,0.0,26420.0,13210.0,13210.0,2021-01-01,1,January,2021
2,Midmarket,France,Carretera,None,2178.0,3,15,32670.0,0.0,32670.0,21780.0,10890.0,2021-06-01,6,June,2021
3,Midmarket,Germany,Carretera,None,888.0,3,15,13320.0,0.0,13320.0,8880.0,4440.0,2021-06-01,6,June,2021
4,Midmarket,Japan,Carretera,None,2470.0,3,15,37050.0,0.0,37050.0,24700.0,12350.0,2021-06-01,6,June,2021


In [4]:
# Get unique values from any particular column
column_name = "Country"
unique_values = data[column_name].unique()
unique_values

array(['Canada', 'Germany', 'France', 'Japan', 'United States of America'],
      dtype=object)

In [5]:
# Query/Filter dataframe, example:
data.query("Country=='Japan'").head(8)

,Segment,Country,Product,Discount Band,Units Sold,Manufacturing Price,Sale Price,Gross Sales,Discounts,Sales,COGS,Profit,Date,Month Number,Month Name,Year
4,Midmarket,Japan,Carretera,None,2470.0,3,15,37050.0,0.0,37050.0,24700.0,12350.0,2021-06-01,6,June,2021
10,Midmarket,Japan,Montana,None,2470.0,5,15,37050.0,0.0,37050.0,24700.0,12350.0,2021-06-01,6,June,2021
12,Small Business,Japan,Montana,None,958.0,5,300,287400.0,0.0,287400.0,239500.0,47900.0,2021-08-01,8,August,2021
17,Midmarket,Japan,Paseo,None,974.0,10,15,14610.0,0.0,14610.0,9740.0,4870.0,2021-02-01,2,February,2021
21,Government,Japan,Paseo,None,883.0,10,7,6181.0,0.0,6181.0,4415.0,1766.0,2021-08-01,8,August,2021
23,Small Business,Japan,Paseo,None,788.0,10,300,236400.0,0.0,236400.0,197000.0,39400.0,2020-09-01,9,September,2020
24,Midmarket,Japan,Paseo,None,2472.0,10,15,37080.0,0.0,37080.0,24720.0,12360.0,2021-09-01,9,September,2021
31,Government,Japan,Velo,None,1493.0,120,7,10451.0,0.0,10451.0,7465.0,2986.0,2021-01-01,1,January,2021


In [6]:
# Query/Filter the dataframe and export the filtered dataframe as an Excel file
for unique_value in unique_values:
    data_output = data.query(f"{column_name} == @unique_value & Year==2021")
    output_path = ATTACHMENT_DIR / f"{unique_value}_2021.xlsx"
    data_output.to_excel(output_path, sheet_name=unique_value, index=False)

## 3. Send Outlook Email with attachment

In [7]:
# Load email distribution list into dataframe
email_list = pd.read_excel(EXCEL_FILE_PATH, sheet_name="Email_List")
email_list

,Country,Name,Email,CC
0,Canada,Emma,emma@codingisfun.com;peter@codingisfun.com,sven@codingisfun.com
1,France,Leo,leo@codingisfun.com,sven@codingisfun.com
2,Germany,Lisa,lisa@codingisfun.com,sven@codingisfun.com
3,Japan,Tanaka-san,tanaka@codingisfun.com,sven@codingisfun.com
4,United States of America,Noah,noah@codingisfun.com,sven@codingisfun.com


In [8]:
# Iterate over email distribution list & send emails via Outlook App
outlook = win32.Dispatch("outlook.application")
for index, row in email_list.iterrows():
    mail = outlook.CreateItem(0)
    mail.To = row["Email"]
    mail.CC = row["CC"]
    mail.Subject = f"Financial Report for: {row['Country']}"
    # mail.Body = "Message body"
    mail.HTMLBody = f"""
                    <b>Hi {row['Name']}</b>,<br><br>
                    Please find attached the report for {row['Country']}.<br><br>
                    Best Regards,<br>
                    Sven
                    """
    attachment_path = str(ATTACHMENT_DIR / f"{row['Country']}_2021.xlsx")
    mail.Attachments.Add(Source=attachment_path)

    mail.Display()
    
    # Uncomment to send email
    # mail.Send()

## 4. Some Remarks

- `OpenPyXL` (&`Pandas`) will not save/remember the formatting of the respective worksheet (colors, font styling, conditional formatting, ..)
- This was a simplified example. In real-life, your dataset might look way messier